<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6d4612dca77df5ec98b9bb9e2a2d2ca95d13ff04b6b48ab79e7a0d77cd593389
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-28 13:08:06
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  9.00 L
Current:  1.37 C
-------------------
Today PnL: -55.85 K (-0.41%)
Current PnL: -20.44 L (-13.75%)
CY Booked + Current PnL: -11.40 L (-7.67%)
-------------------
Total profit:  2.85 L
Total loss:  -23.29 L
-------------------
Total Booked + Current PnL: 15.58 L (12.8%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.61%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.16 C
Est FTT PnL: 78.83 L (57.62%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.45%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-18.25,56.0,H-LC,7.48,112480.0,8168.0,8481.0,-0.38,7.83,7.54,15.96,9.0,0.96,0.88,26.05,X5K,ATH,METALS
53,MEDANTA,1486.00,14.25,63.0,H-MC,14.81,163096.0,36896.0,9280.0,0.43,29.24,5.69,36.59,108.0,3.98,1.28,45.47,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,8285.00,-14.53,61.0,H-LC,8.28,170874.0,14697.0,11397.0,-0.34,9.41,6.67,16.71,22.0,1.29,1.34,28.49,X40N,BTT,HEALTHCARE
66,SBIN,863.00,-13.25,44.0,M-LC,4.07,213921.0,12442.0,14782.0,-0.07,6.18,6.91,13.51,60.0,0.84,1.67,19.55,XY25,NTT,BANKS
81,TTKPRESTIG,770.00,97.87,57.0,M-SC,8.41,85248.0,-15529.0,15626.0,1.66,-15.41,18.33,0.09,245.0,-0.99,0.67,12.00,OX40N,NTT,DURABLES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,5211.76,-9.59,35.0,H-SC,6.14,102878.0,874.0,43054.0,-1.40,0.86,41.85,43.06,122.0,0.02,0.8,18.12,X40N,ATH,MISC
59,RAJOOENG,143.33,-32.60,38.0,H-SC,9.55,102200.0,-300.0,41125.0,-1.19,-0.29,40.24,39.83,114.0,-0.01,0.8,1.19,AR,ATH,MISC


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.49,32.0,H-LC,13.16,195156.0,-21363.0,114264.0,-0.36,-9.87,58.55,42.91,7.0,-0.19,1.53,2.20,AR,NTT,ELECTRICAL
84,UNITDSPR,1644.00,-15.66,36.0,H-LC,8.75,169633.0,-12126.0,47378.0,-1.36,-6.67,27.93,19.39,86.0,-0.26,1.33,0.93,X40N,NTT,BREWERIES
33,HCLTECH,1943.91,-0.22,37.0,H-LC,10.38,176949.0,-13697.0,60216.0,-2.84,-7.18,34.03,24.40,8.0,-0.23,1.38,7.61,X40,ATH,IT
45,ITC,452.00,-39.19,38.0,H-LC,3.13,146219.0,-3713.0,18307.0,-0.41,-2.48,12.52,9.74,4.0,-0.20,1.14,3.61,X40,NTT,FMCG
47,JIOFIN,387.00,-0.75,41.0,H-LC,15.43,219520.0,2114.0,51390.0,-0.68,0.97,23.41,24.61,48.0,0.04,1.72,56.32,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.2,-7.23,46.0,H-LC,3.11,183816.0,-16494.0,76467.0,-1.15,-8.23,41.60,29.94,16.0,-0.22,1.44,26.99,X200,ATH,IT
45,ITC,452.0,-39.19,38.0,H-LC,3.13,146219.0,-3713.0,18307.0,-0.41,-2.48,12.52,9.74,4.0,-0.20,1.14,3.61,X40,NTT,FMCG
55,NESTLEIND,1377.0,-11.41,54.0,H-LC,3.17,272610.0,7184.0,49615.0,0.24,2.71,18.20,21.40,11.0,0.14,2.13,10.24,XY25,NTT,FMCG
20,CIPLA,1795.0,-18.75,61.0,H-LC,4.25,216953.0,12453.0,28963.0,0.24,6.09,13.35,20.25,10.0,0.43,1.70,15.35,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-12.54,55.0,H-LC,6.74,170476.0,-30910.0,118532.0,1.03,-15.35,69.53,43.51,27.0,-0.26,1.33,18.67,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HONAUT,58357.33,-17.33,57.0,H-SC,3.19,116430.0,-11508.0,58646.0,2.43,-9.00,50.37,36.84,143.0,-0.20,0.91,19.20,X40N,ATH,ELECTRICAL
75,TANLA,1963.11,-27.72,51.0,H-SC,8.08,165330.0,-74085.0,347044.0,2.03,-30.94,209.91,114.01,133.0,-0.21,1.29,51.63,AR,ATH,IT
14,BLUESTARCO,2326.38,9.94,60.0,H-MC,4.56,189860.0,25190.0,42775.0,1.80,15.30,22.53,41.28,89.0,0.59,1.49,24.50,X40N,ATH,AC
81,TTKPRESTIG,770.00,97.87,57.0,M-SC,8.41,85248.0,-15529.0,15626.0,1.66,-15.41,18.33,0.09,245.0,-0.99,0.67,12.00,OX40N,NTT,DURABLES
74,SYMPHONY,1306.00,-41.90,31.0,M-SC,17.02,123913.0,-47228.0,47174.0,1.58,-27.60,38.07,-0.03,196.0,-1.00,0.97,1.58,OX40N,NTT,DURABLES


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,1943.91,-0.22,37.0,H-LC,10.38,176949.0,-13697.0,60216.0,-2.84,-7.18,34.03,24.40,8.0,-0.23,1.38,7.61,X40,ATH,IT
67,SFL,1287.00,36.98,55.0,M-SC,10.04,180794.0,-81424.0,151252.0,-2.77,-31.05,83.66,26.63,229.0,-0.54,1.41,12.41,XY24,NTT,MISC
40,INDIAMART,4850.92,-50.91,49.0,H-SC,12.42,136899.0,13563.0,120197.0,-2.70,11.00,87.80,108.45,119.0,0.11,1.07,35.81,AR,ATH,MISC
61,REPCOHOME,880.00,-62.74,23.0,H-SC,7.22,138683.0,-66315.0,211561.0,-2.61,-32.35,152.55,70.85,134.0,-0.31,1.09,13.93,XY24,NTT,FINANCE
30,HAPPSTMNDS,1488.71,-15.86,33.0,H-SC,9.30,59535.0,-37044.0,96780.0,-2.51,-38.36,162.56,61.85,132.0,-0.38,0.47,4.97,AR,ATH,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,97.87,57.0,M-SC,8.41,85248.0,-15529.0,15626.0,1.66,-15.41,18.33,0.09,245.0,-0.99,0.67,12.00,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,80.86,38.0,H-SC,15.16,113994.0,-28485.0,48003.0,-1.33,-19.99,42.11,13.70,163.0,-0.59,0.89,34.54,OX40N,NTT,BANKS
48,KANSAINER,340.0,-69.07,46.0,H-SC,3.37,213381.0,-56286.0,92629.0,1.29,-20.87,43.41,13.47,138.0,-0.61,1.67,7.93,XY24,NTT,PAINTS
41,INDIGOPNTS,1408.0,109.82,41.0,M-SC,1.97,137454.0,-37105.0,37140.0,0.43,-21.26,27.02,0.02,221.0,-1.00,1.08,19.56,OX40N,NTT,PAINTS
19,CERA,9475.0,-21.24,38.0,H-SC,3.02,112383.0,-31420.0,58169.0,-1.09,-21.85,51.76,18.60,149.0,-0.54,0.88,24.02,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-26.76,49.0,H-LC,15.74,242385.0,-53549.0,147467.0,-1.57,-18.10,60.84,31.73,1.0,-0.36,1.90,3.48,X40,BTT,IT
43,INFY,2275.00,-17.80,51.0,H-LC,11.82,268858.0,7257.0,138354.0,-1.80,2.77,51.46,55.67,3.0,0.05,2.10,9.05,X40,BTT,IT
45,ITC,452.00,-39.19,38.0,H-LC,3.13,146219.0,-3713.0,18307.0,-0.41,-2.48,12.52,9.74,4.0,-0.20,1.14,3.61,X40,NTT,FMCG
87,VBL,671.64,-11.31,49.0,H-LC,8.53,269511.0,4471.0,91822.0,-1.49,1.69,34.07,36.33,5.0,0.05,2.11,15.21,X40N,ATH,FMCG
1,ABB,7934.00,-42.49,32.0,H-LC,13.16,195156.0,-21363.0,114264.0,-0.36,-9.87,58.55,42.91,7.0,-0.19,1.53,2.20,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,14.25,63.0,H-MC,14.81,163096.0,36896.0,9280.0,0.43,29.24,5.69,36.59,108.0,3.98,1.28,45.47,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-7.68,72.0,H-LC,11.93,230452.0,45971.0,24405.0,-0.28,24.92,10.59,38.14,38.0,1.88,1.80,38.17,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,14.25,63.0,H-MC,14.81,163096.0,36896.0,9280.0,0.43,29.24,5.69,36.59,108.0,3.98,1.28,45.47,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,9.94,60.0,H-MC,4.56,189860.0,25190.0,42775.0,1.80,15.30,22.53,41.28,89.0,0.59,1.49,24.50,X40N,ATH,AC
88,VOLTAS,1918.49,-2.90,56.0,H-MC,6.54,203700.0,11958.0,84067.0,0.79,6.24,41.27,50.08,99.0,0.14,1.59,13.46,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-3.00,72.0,H-MC,14.96,179837.0,28924.0,28378.0,1.28,19.17,15.78,37.97,93.0,1.02,1.41,49.32,AR,ATH,AUTO
24,DABUR,735.00,-7.44,51.0,H-MC,3.82,201630.0,4602.0,85027.0,-1.03,2.34,42.17,45.49,102.0,0.05,1.58,15.68,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-3.00,72.0,H-MC,14.96,179837.0,28924.0,28378.0,1.28,19.17,15.78,37.97,93.0,1.02,1.41,49.32,AR,ATH,AUTO
40,INDIAMART,4850.92,-50.91,49.0,H-SC,12.42,136899.0,13563.0,120197.0,-2.70,11.00,87.80,108.45,119.0,0.11,1.07,35.81,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-42.68,46.0,M-SC,5.64,96968.0,5470.0,73279.0,0.62,5.98,75.57,86.07,223.0,0.07,0.76,40.27,XR,NTT,DURABLES
90,WIPRO,420.00,-11.52,52.0,M-LC,7.78,155403.0,4458.0,104990.0,-0.55,2.95,67.56,72.51,53.0,0.04,1.22,9.15,XR,NTT,IT
59,RAJOOENG,143.33,-32.60,38.0,H-SC,9.55,102200.0,-300.0,41125.0,-1.19,-0.29,40.24,39.83,114.0,-0.01,0.80,1.19,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,64.0,L-SC,28.09,82080.0,-31469.0,71516.0,-0.49,-27.71,87.13,35.27,268.0,-0.44,0.64,100.89,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-5.35,39.0,H-SC,16.26,88159.0,-12804.0,107334.0,-0.85,-12.68,121.75,93.63,148.0,-0.12,0.69,31.31,SR,ATH,CHEMICALS
52,MASFIN,398.61,-18.25,50.0,H-SC,12.64,93045.0,-4935.0,26536.0,0.91,-5.04,28.52,22.05,152.0,-0.19,0.73,35.25,XR,ATH,FINANCE
35,HINDALCO,756.01,-18.25,56.0,H-LC,7.48,112480.0,8168.0,8481.0,-0.38,7.83,7.54,15.96,9.0,0.96,0.88,26.05,X5K,ATH,METALS
82,UJJIVANSFB,60.00,80.86,38.0,H-SC,15.16,113994.0,-28485.0,48003.0,-1.33,-19.99,42.11,13.70,163.0,-0.59,0.89,34.54,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,64.0,L-SC,28.09,82080.0,-31469.0,71516.0,-0.49,-27.71,87.13,35.27,268.0,-0.44,0.64,100.89,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,-3.00,72.0,H-MC,14.96,179837.0,28924.0,28378.0,1.28,19.17,15.78,37.97,93.0,1.02,1.41,49.32,AR,ATH,AUTO
27,DMART,5201.00,-7.68,72.0,H-LC,11.93,230452.0,45971.0,24405.0,-0.28,24.92,10.59,38.14,38.0,1.88,1.80,38.17,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,9.94,60.0,H-MC,4.56,189860.0,25190.0,42775.0,1.80,15.30,22.53,41.28,89.0,0.59,1.49,24.50,X40N,ATH,AC
6,ASIANTILES,137.00,6700.00,48.0,L-SC,6.37,74628.0,-19182.0,95800.0,-0.79,-20.45,128.37,81.67,269.0,-0.20,0.58,44.52,XR,NTT,CERAMICS


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.91
1,25,42.05
2,50,72.23


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.31
LC    34.52
MC    27.18
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.20
X40N     14.42
X40      14.11
XY25     10.03
XR        9.26
AR        8.75
OX40N     7.61
X5K       2.33
MH        1.73
X200      1.44
SR        1.13
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.05
H-MC    24.21
H-SC    23.12
M-SC    13.60
M-LC     7.39
M-MC     2.64
L-SC     1.59
L-LC     1.08
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.91,-1.82,35.52
IT,12.39,-17.89,76.77
BANKS,7.61,-17.05,66.76
FINANCE,7.49,-26.06,72.54
MISC,6.04,-18.85,86.97
PAINTS,5.76,-18.53,41.90
ELECTRICAL,4.90,-12.31,60.94
AUTO,4.73,-11.93,60.63
AC,3.84,1.33,32.09


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2906856.0,23
XR,1108888.0,13
AR,1091361.0,10
X40,796944.0,10
X40N,524055.0,11
OX40N,486616.0,10
XY25,457288.0,7
SR,199046.0,2
X5K,141408.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2818992.0,27
M-SC,1508695.0,17
H-MC,1284695.0,18
H-LC,1132068.0,17
M-LC,435108.0,5
M-MC,340053.0,2
L-SC,257869.0,3
L-MC,60005.0,1
L-LC,45801.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      953973.0      6
M-SC       XY24      883933.0      7
H-SC       AR        665790.0      5
           XR        531500.0      6
H-MC       XY24      495098.0      5
H-LC       X40       482876.0      5
M-MC       XY24      340053.0      2
H-LC       AR        237022.0      2
H-MC       X40       229008.0      4
H-SC       OX40N     212590.0      4
H-LC       X40N      203965.0      5
H-MC       XY25      199746.0      2
H-SC       SR        199046.0      2
           X40N      194814.0      3
M-LC       XY24      182409.0      2
L-SC       XR        167316.0      2
M-SC       AR        160171.0      2
M-LC       X5K       132927.0      1
M-SC       OX40N     130551.0      4
H-MC       X40N      125276.0      3
M-SC       XY25      125092.0      1
           XR        123888.0      2
H-MC       XR        121189.0      1
M-LC       XR        104990.0      1
L-SC       OX40N      90553.0      1
M-SC       X40        85060.0      1
H-LC       X200       76467.0      1
           XY25       71867.0      2
H-SC       MH         61279.0      1
L-MC       XR         60005.0      1
H-MC       OX40N      52922.0      1
H-LC       XY24       51390.0      1
L-LC       XY25       45801.0      1
H-MC       MH         33078.0      1
           AR         28378.0      1
M-LC       XY25       14782.0      1
H-LC       X5K         8481.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
